In [1]:
import time

import kerasncp as kncp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from keras.layers import (
    RNN,
    Activation,
    Conv2D,
    Dense,
    Dropout,
    Flatten,
    Input,
    InputLayer,
    MaxPool2D,
    TimeDistributed,
    LSTM
)
from keras.models import Sequential
from keras.utils import plot_model
from kerasncp.tf import LTCCell

# from kerasncp.ltc_cell import LTCCell
from kerasncp.wirings import NCP
from ncps import wirings
from ncps.tf import LTC
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam

from config import GlobalConfig
from data_3D import CARLA_Data

NAME = "LTC-Carla-{}".format(int(time.time()))
tensorboard = TensorBoard(
    log_dir="logs/{}".format(NAME),
    write_graph=True,
    update_freq="batch",
    histogram_freq=1,
)
checkpoint_callback = ModelCheckpoint(
    filepath="models/LTC_CNN2D_model-{val_accuracy:.4f}.h5",
    monitor="val_accuracy",
    save_best_only=True,
    save_weights_only=False,
    mode="max",
    verbose=1,
)

In [2]:
def LTC_time_model():
    ncp_wiring = kncp.wirings.NCP(
        inter_neurons=20,  # Number of inter neurons
        command_neurons=10,  # Number of command neurons
        motor_neurons=5,  # Number of motor neurons
        sensory_fanout=4,  # How many outgoing synapses has each sensory neuron
        inter_fanout=5,  # How many outgoing synapses has each inter neuron
        recurrent_command_synapses=6,  # Now many recurrent synapses are in the
        # command neuron layer
        motor_fanin=4,  # How many incoming synapses has each motor neuron
    )
    ncp_cell = LTCCell(
        ncp_wiring,
        initialization_ranges={
            # Overwrite some of the initialization ranges
            "w": (0.2, 2.0),
        },
    )
    model = Sequential()
    
    model.add(InputLayer(input_shape=(None, 160, 704, 3)))
    model.summary()
    model.add(TimeDistributed(Conv2D(24, (5, 5), (2, 2), activation="relu",padding="same")))
    model.summary()
    model.add(TimeDistributed(Conv2D(36, (5, 5), (2, 2), activation="relu")))
    model.add(TimeDistributed(Conv2D(48, (5, 5), (2, 2), activation="relu")))
    model.add(TimeDistributed(Conv2D(64, (5, 5), (2, 2), activation="relu")))
    model.add(TimeDistributed(Conv2D(64, (5, 5), (2, 2), activation="relu")))
    model.add(TimeDistributed(Flatten()))
    model.add(TimeDistributed(Dense(32, activation="relu")))
    model.add(RNN(ncp_cell, return_sequences=True))
    model.add(TimeDistributed(Activation("softmax")))
    model.compile(
        optimizer=Adam(0.01),
        loss="mse",
    )
    return model

def plot_wiring():
    sns.set_style("white")
    plt.figure(figsize=(30, 25))
    legend_handles = ncp_cell.draw_graph(layout='shell',neuron_colors={"command": "tab:cyan"})
    plt.legend(handles=legend_handles, loc="upper center", bbox_to_anchor=(1, 1))
    sns.despine(left=True, bottom=True)
    plt.tight_layout()
    plt.show()

In [4]:
config = GlobalConfig(setting="02_05_withheld")

model = LTC_time_model()

histories = []

epochs = 300

for epoch in range(epochs):
    train_set = CARLA_Data(
        root=config.train_data, current_epoch=epoch, config=config
    ).create_dataset()
    val_set = CARLA_Data(
        root=config.val_data, current_epoch=epoch, config=config
    ).create_dataset()

    
    """ train_metrics = model.evaluate_generator(val_set, steps=None, verbose=0)
    print("Train Loss:", train_metrics[0])
    print("Train Metrics (if applicable):", train_metrics[1:])   """
        

    history = model.fit(
        train_set,
        epochs=1,
        validation_data=val_set,
        callbacks=[tensorboard, checkpoint_callback],
    )

Skip Town02 and Town05
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_8 (TimeDis  (None, None, 80, 352, 24  1824     
 tributed)                   )                                   
                                                                 
Total params: 1,824
Trainable params: 1,824
Non-trainable params: 0
_________________________________________________________________
      1/Unknown - 18s 18s/step - loss: 0.0814